# Задача на 2-ом шаге урока

**🧠 Практика по apply() на 3 столбцах**

Датасет для тестирования:

Задание: Создайте в df новый столбец final_ride_cost, который будет зависеть от рейтинга (rating) и максимальной скорости (speed_max) поездки. Значения в этом столбце будут отличаться в зависимости от двух условий:
- Если рейтинг выше 6 и максимальная скорость поездки меньше 120 км/ч, то стоимость поездки (ride_cost) уменьшите на 5%.
- В противном случае: стоимость поездки (ride_cost) увеличьте на 5%.

В новом столбце final_ride_cost отбросьте дробную часть значений, оставьте только целую часть. Так как стоимость измеряется в рублях, дробная часть не имеет значения.

In [935]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/3_1_2.csv')

# В столбце true_final_ride_cost записаны правильные ответы.
# Создайте столбец final_ride_cost исходя из условий задачи.
df.head()

,user_id,rating,speed_max,ride_cost,true_final_ride_cost
0,J19512539T,6.73,78.000000,4934,4687
1,B22315336q,7.46,72.000000,235,223
2,y82944196A,4.86,168.181029,63835,67026
3,G66226901h,7.70,74.000000,780,741
4,V24342105N,1.09,127.000000,891,935


In [936]:
df['final_ride_cost'] = df.apply(lambda x: x['ride_cost'] * 0.95 
                                 if (x['rating'] > 6) and (x['speed_max'] < 120)  
                                 else x['ride_cost'] * 1.05, 
                                 axis=1).astype(np.int64)

In [937]:
df

,user_id,rating,speed_max,ride_cost,true_final_ride_cost,final_ride_cost
0,J19512539T,6.73,78.000000,4934,4687,4687
1,B22315336q,7.46,72.000000,235,223,223
2,y82944196A,4.86,168.181029,63835,67026,67026
3,G66226901h,7.70,74.000000,780,741,741
4,V24342105N,1.09,127.000000,891,935,935
...,...,...,...,...,...,...
95,W34824642c,1.55,159.094786,1704,1789,1789
96,y70362364p,4.70,94.596531,333,349,349
97,k20669275M,1.18,50.752477,294,308,308
98,h12083929g,4.73,111.892026,2940,3087,3087


In [938]:
# Корректность своего ответа можно проверить, например, таким образом
print(df['final_ride_cost'].equals(df['true_final_ride_cost']))

True


На вход подается: Переменная df  с полным DataFrame с исходными данными.
На выходе принимается: Измененный DataFrame (переменная df).

# Задача на 3-ем шаге урока

**🧠 Задача: 🖐 gold фичей**

Задача:

Создайте новый датафрейм, в котором для каждой машины есть следующие столбцы:

- Столбец feature_1 - Количество уникальных значений остановок (stop_times).
- Столбец feature_2 - Второй максимум столбца максимальной скорости (speed_max).
- Столбец feature_3 - Отношение минимального и максимального значения рейтинга.
- Столбец feature_4 - Самое популярное значение числа остановок (другим словами, мода).
- Столбец feature_5 - Стандартное отклонение стоимостей поездок (ride_cost).

Решение запишите в переменную result.

In [939]:
import pandas as pd

df = pd.read_csv('../data/3_1_3.csv')

df.head(3)

,car_id,rating,ride_duration,ride_cost,speed_max,stop_times
0,G-1056464Y,3.06,8,106,57.000000,1
1,G-1056464Y,4.66,870,8694,96.000000,3
2,G-1056464Y,7.37,16,202,58.690905,1


In [940]:
df['speed_max'].nlargest(2).values[1]

166.90812458034512

In [941]:
f2 = lambda x: x.nlargest(2).values[1]
f3 = lambda x: x.min() / x.max()
f4 = lambda x: x.mode()[0]

result = df.groupby('car_id', as_index=False).agg(
    feature_1=('stop_times', 'nunique'),
    feature_2=('speed_max', f2),
    feature_3=('rating', f3),
    feature_4=('stop_times', f4),
    feature_5=('ride_cost', 'std')
)

In [942]:
result.head(3)

,car_id,feature_1,feature_2,feature_3,feature_4,feature_5
0,G-1056464Y,5,96.000000,0.299279,1,125833.828153
1,M15952724y,4,151.000000,0.054645,0,26887.711194
2,M50269450O,8,115.597945,0.241451,0,136560.952417


In [943]:
# В переменной ans находится правильный ответ для df
ans = pd.read_csv('../data/3_1_3_ans.csv')
ans.head(3)

,car_id,feature_1,feature_2,feature_3,feature_4,feature_5
0,G-1056464Y,5,96.000000,0.299279,1,125833.828153
1,M15952724y,4,151.000000,0.054645,0,26887.711194
2,M50269450O,8,115.597945,0.241451,0,136560.952417


На вход подается: Переменная df  с исходными данными.
На выходе принимается: Переменная result.

# Задача на 4-ом шаге урока

**🧠 Задача на merge()**

Задача:
- Обогатите датасет rides про поездки информацией о будущем виде поломки, используя датасет cars.
- Удалите лишние столбцы. Таблицу rides надо дополнить лишь столбцом target_class.
- Удалите поездки, для которых нет информации про поломки.  
- Запишите в переменную result.


In [944]:
import pandas as pd

rides = pd.read_csv('../data/3_1_4_rides.csv')
cars = pd.read_csv('../data/3_1_4_cars.csv')

display(rides.head(3), cars.head(3))

,car_id,ride_id,rating,ride_duration,ride_cost
0,b14467388K,Z1C,5.03,22,237
1,d-1627526q,a1x,1.14,131,1567
2,S16201816J,x1Q,6.19,115,1833


,car_id,model,riders,target_class
0,s-1723833v,BMW 320i,58695,engine_check
1,E16896376o,VW Polo VI,89960,engine_overheat
2,X10754765V,VW Tiguan,125986,electro_bug


На выходе принимается: Переменная result

In [945]:
result = pd.merge(left=rides, right=cars[['car_id', 'target_class']], on='car_id', how='inner')
result.head(3)

,car_id,ride_id,rating,ride_duration,ride_cost,target_class
0,L-1156474z,u1g,5.81,33,391,break_bug
1,M20797256F,e1V,5.01,56,667,engine_overheat
2,E16896376o,B1Q,4.88,26,228,engine_overheat


In [946]:
# В переменную ans записан правильный ответ, с которым можно сверить свое решение.
ans = pd.read_csv('../data/3_1_4_ans.csv')
ans.head(3)

,car_id,ride_id,rating,ride_duration,ride_cost,target_class
0,L-1156474z,u1g,5.81,33,391,break_bug
1,M20797256F,e1V,5.01,56,667,engine_overheat
2,E16896376o,B1Q,4.88,26,228,engine_overheat


# Задача на 5-ом шаге урока

**🧠 Задача на transform() и 3 silver фичи**

На вход подается: Переменная rides с исходными данными про поездки.

Задача:

- Обогатите датасет rides про поездки дополнительными признаками без использования метода merge.
- Добавьте столбец car_ride_count - общее число поездок для этой машины.
- Добавьте столбец user_rating_max - максимальный рейтинг, поставленный этим пользователем.
- Добавьте столбец user_ride_cost_mean - средняя стоимость поездки этого пользователя.
- Результат запишите в переменную result.

In [947]:
import pandas as pd

rides = pd.read_csv('../data/3_1_5.csv')
rides.head(3)

,car_id,ride_id,user_id,rating,ride_cost,true_car_ride_count,true_user_rating_max,true_user_ride_cost_mean
0,D17587481H,E1V,Q20675647n,1.96,282,3,1.96,282.0
1,w13176062d,V1h,O11803938a,4.45,308,4,4.45,308.0
2,s-2202835E,B1g,u20234217o,2.76,525,4,2.76,525.0


In [948]:
# Делаем группировку
rides_gr = rides.groupby('car_id', as_index=False).agg(rating_mean = ('rating', 'mean')) 

# Приклеиваем полученные значения к исходной таблице
rides = rides.merge(rides_gr, on = 'car_id', how = 'left') 

# Смотрим результат
rides.head(5)

,car_id,ride_id,user_id,rating,ride_cost,true_car_ride_count,true_user_rating_max,true_user_ride_cost_mean,rating_mean
0,D17587481H,E1V,Q20675647n,1.96,282,3,1.96,282.0,5.466667
1,w13176062d,V1h,O11803938a,4.45,308,4,4.45,308.0,3.907500
2,s-2202835E,B1g,u20234217o,2.76,525,4,2.76,525.0,2.272500
3,H-1213278K,i1U,c19828320U,7.12,6154,5,7.12,6154.0,4.176000
4,H-2108091L,j1Z,l90945335K,1.68,539,4,1.68,539.0,3.355000


In [949]:
rides = pd.read_csv('../data/3_1_5.csv')

# В столбцах true_car_ride_count, true_user_rating_max, true_user_ride_cost_mean записаны правильные ответы.
# Создайте столбцы car_ride_count, user_rating_max, user_ride_cost_mean

rides['car_ride_count'] = rides.groupby('car_id', as_index=False)['ride_id'].transform('count')
rides['user_rating_max'] = rides.groupby('user_id', as_index=False)['rating'].transform('max')
rides['user_ride_cost_mean'] = rides.groupby('user_id', as_index=False)['ride_cost'].transform('mean')
rides.head(3)

,car_id,ride_id,user_id,rating,ride_cost,true_car_ride_count,true_user_rating_max,true_user_ride_cost_mean,car_ride_count,user_rating_max,user_ride_cost_mean
0,D17587481H,E1V,Q20675647n,1.96,282,3,1.96,282.0,3,1.96,282.0
1,w13176062d,V1h,O11803938a,4.45,308,4,4.45,308.0,4,4.45,308.0
2,s-2202835E,B1g,u20234217o,2.76,525,4,2.76,525.0,4,2.76,525.0


In [950]:
# Проверить корректность своего решения можно, например, таким способом
print(rides['true_car_ride_count'].equals(rides['car_ride_count']))
print(rides['true_user_rating_max'].equals(rides['user_rating_max']))
print(rides['true_user_ride_cost_mean'].equals(rides['user_ride_cost_mean']))

True
True
True


In [951]:
result = rides

На выходе принимается: Переменная result.

# Задача на 6-ом шаге урока


**🧠 Задача на transform() и 4 самописные функции**

На вход подается: Переменная rides  с исходными данными про поездки.

Задача

- Обогатите датасет rides про поездки дополнительными признаками без использования метода merge.
- Добавьте столбец feature_1 — 75% перцентиль столбца ride_cost для каждого пользователя (user_id).
- 🎃 Добавьте столбец feature_2 — разницу максимального и минимального значения rating, деленную на среднее столбца rating для каждого пользователя (user_id).
- 👽 Добавьте столбец feature_3  — количество значений ride_duration, которое больше 40 для каждой машины (car_id).
- 👾 Добавьте столбец feature_4 — сумму значений ride_duration, которая больше 40 для каждой машины (car_id).
- Результат запишите в переменную result.


In [952]:
import pandas as pd

rides = pd.read_csv('../data/3_1_6.csv')
rides.head(10)

,car_id,ride_id,user_id,rating,ride_duration,ride_cost,true_feature_1,true_feature_2,true_feature_3,true_feature_4
0,v13590850L,m1q,S54951221d,1.38,52,723,599.25,1.043328,1,52
1,C-1092574s,q1w,h90416721m,8.36,33572,470001,470001.00,0.000000,6,100329
2,i77678921h,i1i,A15424124M,7.03,49,385,385.00,0.000000,3,169
3,U16614632F,p1m,Q22613814X,6.87,43,510,510.00,0.000000,5,320
4,i55328849i,I1A,N22883211D,4.30,110,1536,1536.00,0.000000,10,23108
5,C-1092574s,k1Q,r18933707Y,0.10,19,185,185.00,0.000000,6,100329
6,Y97882055f,S1K,b61297238i,3.44,125,1119,836.00,1.875283,1,125
7,i55328849i,V1L,Y16580421n,5.57,76,680,680.00,0.000000,10,23108
8,s20690018N,w1s,P60842443a,3.90,16,138,629.25,0.190255,0,0
9,d71004474f,M1o,d18276261h,7.59,128,1275,1275.00,0.000000,1,128


In [953]:
# В столбцы true_feature_1, true_feature_2, true_feature_3, true_feature_4 записаны правильные ответы
# Создайте столбцы feature_1, feature_2, feature_3, feature_4.

f1 = lambda x: np.percentile(x, 75)
f2 = lambda x: (x.max() - x.min()) / x.mean()
f3 = lambda x: np.sum(x > 40)
f4 = lambda x: np.sum(x[x > 40])

rides['feature_1'] = rides.groupby('user_id', as_index=False)['ride_cost'].transform(f1)
rides['feature_2'] = rides.groupby('user_id', as_index=False)['rating'].transform(f2)
rides['feature_3'] = rides.groupby('car_id', as_index=False)['ride_duration'].transform(f3)
rides['feature_4'] = rides.groupby('car_id')['ride_duration'].transform(f4)

rides.head(10)

,car_id,ride_id,user_id,rating,ride_duration,ride_cost,true_feature_1,true_feature_2,true_feature_3,true_feature_4,feature_1,feature_2,feature_3,feature_4
0,v13590850L,m1q,S54951221d,1.38,52,723,599.25,1.043328,1,52,599.25,1.043328,1,52
1,C-1092574s,q1w,h90416721m,8.36,33572,470001,470001.00,0.000000,6,100329,470001.00,0.000000,6,100329
2,i77678921h,i1i,A15424124M,7.03,49,385,385.00,0.000000,3,169,385.00,0.000000,3,169
3,U16614632F,p1m,Q22613814X,6.87,43,510,510.00,0.000000,5,320,510.00,0.000000,5,320
4,i55328849i,I1A,N22883211D,4.30,110,1536,1536.00,0.000000,10,23108,1536.00,0.000000,10,23108
5,C-1092574s,k1Q,r18933707Y,0.10,19,185,185.00,0.000000,6,100329,185.00,0.000000,6,100329
6,Y97882055f,S1K,b61297238i,3.44,125,1119,836.00,1.875283,1,125,836.00,1.875283,1,125
7,i55328849i,V1L,Y16580421n,5.57,76,680,680.00,0.000000,10,23108,680.00,0.000000,10,23108
8,s20690018N,w1s,P60842443a,3.90,16,138,629.25,0.190255,0,0,629.25,0.190255,0,0
9,d71004474f,M1o,d18276261h,7.59,128,1275,1275.00,0.000000,1,128,1275.00,0.000000,1,128


In [954]:
# Проверить корректность своего решения можно, например, таким способом
print(rides['true_feature_1'].equals(rides['feature_1']))
print(rides['true_feature_2'].round(2).equals((rides['feature_2']).round(2)))
print(rides['true_feature_3'].equals(rides['feature_3']))
print(rides['true_feature_4'].equals(rides['feature_4']))

True
True
True
True


На выходе принимается: Переменная result

In [955]:
result = rides

# Задача на 7-ом шаге урока

**🧠 Задача на pivot_table() и merge()**

In [956]:
car_train = pd.read_csv('../data/car_train.csv')
car_train.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,108.53,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,35.20,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,38.62,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,30.34,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,30.45,engine_fuel


In [957]:
car_train_pivot = car_train.pivot_table(index = 'year_to_start',
                                        columns = ['car_type'],
                                        values = ['riders'],
                                        aggfunc = ['mean', 'sum', 'max']).fillna(0)
car_train_pivot

mean                                              \
                      riders                                               
car_type            business        economy        premium      standart   
year_to_start                                                              
2011             2577.384615    3552.727273       0.000000   3751.494118   
2012            23009.117647   23381.022388       0.000000  23193.769231   
2013            43257.074074   43332.507519   42755.687500  43751.048387   
2014            63187.086957   62482.090909   62991.200000  62754.522727   
2015            81891.451613   83058.953846   83445.875000  82515.107784   
2016           101821.000000  101913.714286  101373.411765      0.000000   
2017           125625.400000  122085.581281  123482.615385      0.000000   

                     sum                                          max  \
                  riders                                       riders   
car_type        business     economy    premium    standart  business   
year_to_start                                                           
2011             33506.0    390800.0        0.0    318877.0    7708.0   
2012            391155.0   3133057.0        0.0   2110633.0   30175.0   
2013           1167941.0  11526447.0   684091.0   5425130.0   49634.0   
2014           1453303.0  17869878.0   629912.0   8283597.0   76277.0   
2015           2538635.0  32392992.0  1335134.0  13780023.0   92250.0   
2016            101821.0  14981316.0  1723348.0         0.0  101821.0   
2017            628127.0  24783373.0  3210548.0         0.0  134530.0   

                                            
                                            
car_type        economy   premium standart  
year_to_start                               
2011            12613.0       0.0  18628.0  
2012            31246.0       0.0  31867.0  
2013            58090.0   54681.0  53682.0  
2014            77167.0   70494.0  76591.0  
2015            99295.0   93044.0  97217.0  
2016           119546.0  111906.0      0.0  
2017           142862.0  139872.0      0.0

Задача. На основе этого датафрейма надо будет собрать несколько сводных таблиц (pivot table) и потом смержить их в одну итоговую, получив некое подобие эмбеддингов для подачи в модель.

- Индексом во всех таблицах будет user_id.
- 🙂 Первая таблица: Сколько раз пользователь брал машины определенного класса?
- 🙃 Вторая таблица: Какими были максимальная, минимальная и средняя дистанция и стоимость поездки пользователя?
- 🙂 Третья таблица: Сколько уникальных машин брал пользователь в аренду?
- Далее соединяем по ключу (смерживаем) все 3 таблицы в одну.
- И в конце добавляем бинарный таргет в столбец crazy_driver: Является ли наш пользователь безумным гонщиком? Если его максимальная скорость превышала 100 км/ч, то 1; если не превышала, то 0.
- Результат решения запишите в переменную result.

In [958]:
import pandas as pd

df = pd.read_csv('../data/3_1_7.csv')
df.head(3)

,user_id,car_type,car_id,distance,ride_cost,speed_max,model
0,h14343699V,economy,A-1049127W,158.842938,135,72.027288,Hyundai Solaris
1,b13580442h,standart,A-1079539w,3288.823484,510,124.758500,Renault Kaptur
2,R21123881y,economy,A-1162143G,1464.013497,584,47.000000,VW Polo


In [959]:
df_pivot_1 = df.pivot_table(index = 'user_id',
                            columns = ['car_type'],
                            values = ['car_id'],
                            aggfunc = ['count']).fillna(0)

df_pivot_1.columns = ['_'.join(i[::-1]) for i in df_pivot_1.columns]
df_pivot_1.head()

,economy_car_id_count,standart_car_id_count
user_id,,
A11173530w,1.0,0.0
A66187474K,0.0,1.0
C84438702e,1.0,0.0
E15036556k,0.0,1.0
G84350611k,0.0,1.0


In [960]:
df_pivot_2 = df[['user_id', 'ride_cost', 'distance']].pivot_table(index = 'user_id',
                            aggfunc = ['min', 'max', 'mean']).fillna(0)

df_pivot_2.columns = ['_'.join(i[::-1]) for i in df_pivot_2.columns]
df_pivot_2.head()

,distance_min,ride_cost_min,distance_max,ride_cost_max,distance_mean,ride_cost_mean
user_id,,,,,,
A11173530w,2.967598e+03,614,2.967598e+03,614,2.967598e+03,614.0
A66187474K,4.621773e+03,1320,4.621773e+03,1320,4.621773e+03,1320.0
C84438702e,1.221070e+03,195,1.221070e+03,195,1.221070e+03,195.0
E15036556k,1.284249e+06,374394,1.284249e+06,374394,1.284249e+06,374394.0
G84350611k,1.850353e+03,428,1.850353e+03,428,1.850353e+03,428.0


In [961]:
df_pivot_3 = df.pivot_table(index = 'user_id',
                            values = ['car_id'],
                            aggfunc = ['nunique']).fillna(0)

df_pivot_3.columns = ['_'.join(i[::-1]) for i in df_pivot_3.columns]
df_pivot_3.head()

,car_id_nunique
user_id,
A11173530w,1
A66187474K,1
C84438702e,1
E15036556k,1
G84350611k,1


In [962]:
result = df_pivot_1.merge(df_pivot_2, on='user_id').merge(df_pivot_3, on='user_id').merge(df[['user_id', 'speed_max']], on='user_id')
result.head()

,user_id,economy_car_id_count,standart_car_id_count,distance_min,ride_cost_min,distance_max,ride_cost_max,distance_mean,ride_cost_mean,car_id_nunique,speed_max
0,A11173530w,1.0,0.0,2.967598e+03,614,2.967598e+03,614,2.967598e+03,614.0,1,128.466724
1,A66187474K,0.0,1.0,4.621773e+03,1320,4.621773e+03,1320,4.621773e+03,1320.0,1,80.400106
2,C84438702e,1.0,0.0,1.221070e+03,195,1.221070e+03,195,1.221070e+03,195.0,1,137.234292
3,E15036556k,0.0,1.0,1.284249e+06,374394,1.284249e+06,374394,1.284249e+06,374394.0,1,62.000000
4,G84350611k,0.0,1.0,1.850353e+03,428,1.850353e+03,428,1.850353e+03,428.0,1,76.000000


In [963]:
# И в конце добавляем бинарный таргет в столбец crazy_driver: 
# Является ли наш пользователь безумным гонщиком? 
# Если его максимальная скорость превышала 100 км/ч, то 1; 
# если не превышала, то 0.
    

result['crazy_driver'] = np.where(result['speed_max'] > 100, 1, 0)
result = result.drop(columns=['speed_max']) \
               .rename(columns={'economy_car_id_count': 'economy_count',
                                'standart_car_id_count': 'standart_count'})
result.head()

,user_id,economy_count,standart_count,distance_min,ride_cost_min,distance_max,ride_cost_max,distance_mean,ride_cost_mean,car_id_nunique,crazy_driver
0,A11173530w,1.0,0.0,2.967598e+03,614,2.967598e+03,614,2.967598e+03,614.0,1,1
1,A66187474K,0.0,1.0,4.621773e+03,1320,4.621773e+03,1320,4.621773e+03,1320.0,1,0
2,C84438702e,1.0,0.0,1.221070e+03,195,1.221070e+03,195,1.221070e+03,195.0,1,1
3,E15036556k,0.0,1.0,1.284249e+06,374394,1.284249e+06,374394,1.284249e+06,374394.0,1,0
4,G84350611k,0.0,1.0,1.850353e+03,428,1.850353e+03,428,1.850353e+03,428.0,1,0


In [964]:
# В ans записан правильный ответ
ans = pd.read_csv('../data/3_1_7_ans.csv')
ans.head()

,user_id,economy_count,standart_count,distance_min,ride_cost_min,distance_max,ride_cost_max,distance_mean,ride_cost_mean,car_id_nunique,crazy_driver
0,A11173530w,1.0,0.0,2.967598e+03,614,2.967598e+03,614,2.967598e+03,614,1,1
1,A66187474K,0.0,1.0,4.621773e+03,1320,4.621773e+03,1320,4.621773e+03,1320,1,0
2,C84438702e,1.0,0.0,1.221070e+03,195,1.221070e+03,195,1.221070e+03,195,1,1
3,E15036556k,0.0,1.0,1.284249e+06,374394,1.284249e+06,374394,1.284249e+06,374394,1,0
4,G84350611k,0.0,1.0,1.850353e+03,428,1.850353e+03,428,1.850353e+03,428,1,0


In [ ]:
# result =

Результат решения запишите в переменную result

Комментарий: Не забывайте избавляться от мультииндекса и использовать pivot_table. Использовать groupby нельзя! Пропуски заполнять 0.